In [3]:
import theano
import theano.tensor as tt
import numpy as np
import pymc3 as pm

In [4]:
from IPython.display import IFrame
IFrame(src='https://docs.pymc.io/PyMC3_and_Theano.html', width=700, height=600)

In [5]:
# We don't specify the dtype of our input variables, so it
# defaults to using float64 without any special config.
a = tt.scalar('a')
x = tt.vector('x')
# `tt.ivector` creates a symbolic vector of integers.
y = tt.ivector('y')

inner = a * x**3 + y**2
out = tt.exp(inner).sum()

In [7]:
func = theano.function([a, x, y], [out], allow_input_downcast=True)

a_val = 1.2
x_vals = np.random.randn(10)
y_vals = np.random.randn(10)

out = func(a_val, x_vals, y_vals)

In [8]:
a = tt.vector('a')
if (a > 0).all():
    b = tt.sqrt(a)
else:
    b = -a

a = tt.vector('a')
b = tt.sqrt(a)

a = tt.vector('a')
b = tt.switch((a > 0).all(), tt.sqrt(a), -a)

a = tt.vector('a')
# Access the 10th element. This will fail when a function build
# from this expression is executed with an array that is too short.
b = a[10]

# Extract a subvector
b = a[[1, 2, 10]]

a = tt.vector('a')
b = tt.set_subtensor(a[:10], 1)

# is roughly equivalent to this (although theano avoids
# the copy if `a` isn't used anymore)
a = np.random.randn(10)
b = a.copy()
b[:10] = 1

In [6]:
true_mu = 0.1
data = true_mu + np.random.randn(50)

with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    y = pm.Normal('y', mu=mu, sigma=1, observed=data)

# For illustration only, those functions don't actually exist
# in exactly this way!
model = pm.Model()

mu = tt.scalar('mu')
model.add_free_variable(mu)
model.add_logp_term(pm.Normal.dist(0, 1).logp(mu))

model.add_logp_term(pm.Normal.dist(mu, 1).logp(data))


with pm.Model() as model:
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd', 1)
    y = pm.Normal('y', mu=mu, sigma=sd, observed=data)


# For illustration only, not real code!
model = pm.Model()
mu = tt.scalar('mu')
model.add_free_variable(mu)
model.add_logp_term(pm.Normal.dist(0, 1).logp(mu))

sd_log__ = tt.scalar('sd_log__')
model.add_free_variable(sd_log__)
model.add_logp_term(corrected_logp_half_normal(sd_log__))

sd = tt.exp(sd_log__)
model.add_deterministic_variable(sd)

model.add_logp_term(pm.Normal.dist(mu, sd).logp(data))

design_matrix = np.array([[...]])
with pm.Model() as model:
    # beta is a tt.dvector
    beta = pm.Normal('beta', 0, 1, shape=len(design_matrix))
    predict = tt.dot(design_matrix, beta)
    sd = pm.HalfCauchy('sd', beta=2.5)
    pm.Normal('y', mu=predict, sigma=sd, observed=data)

AttributeError: 'Model' object has no attribute 'add_free_variable'